In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.tabular import *
from fastai.vision import *

In [3]:
#!kaggle competitions download -c favorita-grocery-sales-forecasting

In [4]:
# import zipfile, os
# path_to_zip_file = 'favorita-grocery-sales-forecasting.zip'
# os.makedirs('data/groceries', exist_ok=True)
# directory_to_extract_to = 'data/groceries/'
# with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
#     zip_ref.extractall(directory_to_extract_to)

In [5]:
# from py7zr import unpack_7zarchive
# import shutil
# shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)
# for f in os.listdir('data/groceries/'):
#     shutil.unpack_archive(f'data/groceries/{f}', 'data/groceries/')

In [6]:
path = Path('data/groceries/'); path

PosixPath('data/groceries')

## Read Data

In [7]:
types = {'id': 'int64',
        'item_nbr': 'int32',
        'store_nbr': 'int8',
        'unit_sales': 'float32',
        'onpromotion': 'object'}

In [8]:
%%time
df_raw = pd.read_csv(f'{path}/train.csv', parse_dates=['date'],
                    dtype=types, infer_datetime_format=True)

CPU times: user 55.4 s, sys: 6.67 s, total: 1min 2s
Wall time: 1min 7s


In [12]:
df_raw.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


In [10]:
df_raw.columns

Index(['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion'], dtype='object')

In [10]:
len(df_raw)

125497040

In [9]:
df_raw.onpromotion.fillna(False, inplace=True)
df_raw.onpromotion = df_raw.onpromotion.map({'False': False,'True': True})
df_raw.onpromotion = df_raw.onpromotion.astype(bool)
df_raw.unit_sales = np.log1p(np.clip(df_raw.unit_sales, 0, None))
df_raw.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,2.079442,True
1,1,2013-01-01,25,105574,0.693147,True
2,2,2013-01-01,25,105575,1.098612,True
3,3,2013-01-01,25,108079,0.693147,True
4,4,2013-01-01,25,108701,0.693147,True


In [13]:
!dir {path}

holidays_events.csv  oil.csv		    stores.csv	train.csv
items.csv	     sample_submission.csv  test.csv	transactions.csv


In [9]:
table_names1 = ['stores', 'items']
tables1 = [pd.read_csv(f'{path}/{f}.csv', low_memory=False) for f in table_names1]
for f in tables1: print(f.head())
stores, items = tables1

   store_nbr           city                           state type  cluster
0          1          Quito                       Pichincha    D       13
1          2          Quito                       Pichincha    D       13
2          3          Quito                       Pichincha    D        8
3          4          Quito                       Pichincha    D        9
4          5  Santo Domingo  Santo Domingo de los Tsachilas    D        4
   item_nbr        family  class  perishable
0     96995     GROCERY I   1093           0
1     99197     GROCERY I   1067           0
2    103501      CLEANING   3008           0
3    103520     GROCERY I   1028           0
4    103665  BREAD/BAKERY   2712           1


In [29]:
table_names = ['holidays_events', 'oil', 'transactions', 'test']
tables = [pd.read_csv(f'{path}/{f}.csv', parse_dates = ['date'], infer_datetime_format=True) for f in table_names]
for f in tables: print(f.head())
holidays, oil, transactions, test = tables
len(tables)

        date     type    locale locale_name                    description  \
0 2012-03-02  Holiday     Local       Manta             Fundacion de Manta   
1 2012-04-01  Holiday  Regional    Cotopaxi  Provincializacion de Cotopaxi   
2 2012-04-12  Holiday     Local      Cuenca            Fundacion de Cuenca   
3 2012-04-14  Holiday     Local    Libertad      Cantonizacion de Libertad   
4 2012-04-21  Holiday     Local    Riobamba      Cantonizacion de Riobamba   

   transferred  
0        False  
1        False  
2        False  
3        False  
4        False  
        date  dcoilwtico
0 2013-01-01         NaN
1 2013-01-02       93.14
2 2013-01-03       92.97
3 2013-01-04       93.12
4 2013-01-07       93.20
        date  store_nbr  transactions
0 2013-01-01         25           770
1 2013-01-02          1          2111
2 2013-01-02          2          2358
3 2013-01-02          3          3487
4 2013-01-02          4          1922
          id       date  store_nbr  item_nbr  onpro

4

In [30]:
def join_df(left, right, left_on, right_on=None, suffix= '_y'):
    if right_on is None:
        right_on = left_on
    return left.merge(right, how='left', left_on = left_on,
                      right_on=right_on, suffixes = ("", suffix))

In [31]:
holidays = holidays.drop(columns = ['locale', 'locale_name', 'description', 'transferred'])
holidays.dtypes

date    datetime64[ns]
type            object
dtype: object

In [14]:
holidays['type'].unique()

array(['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event'], dtype=object)

In [32]:
holidays.rename(columns = {'type':'Holiday'}, inplace = True) # stores table has same column name.
holidays.dtypes

date       datetime64[ns]
Holiday            object
dtype: object

In [15]:
df_raw['date'].max(), df_raw['date'].min

(Timestamp('2017-08-15 00:00:00'),
 <bound method Series.min of 0           2013-01-01
 1           2013-01-01
 2           2013-01-01
 3           2013-01-01
 4           2013-01-01
                ...    
 125497035   2017-08-15
 125497036   2017-08-15
 125497037   2017-08-15
 125497038   2017-08-15
 125497039   2017-08-15
 Name: date, Length: 125497040, dtype: datetime64[ns]>)

In [17]:
holidays['date'].max(), holidays['date'].min()

(Timestamp('2017-12-26 00:00:00'), Timestamp('2012-03-02 00:00:00'))

In [19]:
oil.dtypes

date          datetime64[ns]
dcoilwtico           float64
dtype: object

In [14]:
x = {'dcoilwtico': 'float16'}
oil = oil.astype(x)
oil.dtypes

date          datetime64[ns]
dcoilwtico           float16
dtype: object

In [15]:
# mergeing df_raw with oil
df_raw = join_df(df_raw, oil, 'date')
len(df_raw)

125497040

In [16]:
holidays.head()

,date,Holiday
0,2012-03-02,Holiday
1,2012-04-01,Holiday
2,2012-04-12,Holiday
3,2012-04-14,Holiday
4,2012-04-21,Holiday


In [40]:
#merging df_raw with holidays
df_raw = join_df(df_raw, holidays, 'date')
len(df_raw)

127970257

In [20]:
df_raw['date'].min(), df_raw['date'].max()

(Timestamp('2013-01-01 00:00:00'), Timestamp('2017-08-15 00:00:00'))

In [19]:
df_raw.loc[df_raw['date'] == '2013-01-02'][:5]

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,dcoilwtico,Holiday
578,578,2013-01-02,1,103665,1.098612,True,93.125,NaN
579,579,2013-01-02,1,105574,2.197225,True,93.125,NaN
580,580,2013-01-02,1,105575,2.772589,True,93.125,NaN
581,581,2013-01-02,1,105577,1.098612,True,93.125,NaN
582,582,2013-01-02,1,105737,1.098612,True,93.125,NaN


In [15]:
items.columns

Index(['item_nbr', 'family', 'class', 'perishable'], dtype='object')

In [41]:
# merging df_raw with items
df_raw = join_df(df_raw, items, 'item_nbr')
len(df_raw), df_raw.head()

(127970257,
    id       date  store_nbr  item_nbr  unit_sales onpromotion  dcoilwtico  \
 0   0 2013-01-01         25    103665         7.0         NaN         NaN   
 1   1 2013-01-01         25    105574         1.0         NaN         NaN   
 2   2 2013-01-01         25    105575         2.0         NaN         NaN   
 3   3 2013-01-01         25    108079         1.0         NaN         NaN   
 4   4 2013-01-01         25    108701         1.0         NaN         NaN   
 
    Holiday        family  class  perishable  
 0  Holiday  BREAD/BAKERY   2712           1  
 1  Holiday     GROCERY I   1045           0  
 2  Holiday     GROCERY I   1045           0  
 3  Holiday     GROCERY I   1030           0  
 4  Holiday          DELI   2644           1  )

In [42]:
df_raw.columns

Index(['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion',
       'dcoilwtico', 'Holiday', 'family', 'class', 'perishable'],
      dtype='object')

In [50]:
len(stores), len(transactions)

(54, 83488)

In [43]:
# merging stores and transactions
stores_t = join_df(stores, transactions, 'store_nbr')
len(stores_t)

83488

In [44]:
stores_t.rename(columns = {'type':'StoreType'}, inplace = True)
stores_t.columns

Index(['store_nbr', 'city', 'state', 'StoreType', 'cluster', 'date',
       'transactions'],
      dtype='object')

In [45]:
df_raw.columns

Index(['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion',
       'dcoilwtico', 'Holiday', 'family', 'class', 'perishable'],
      dtype='object')

In [46]:
# merging stores_t with df_raw
df_raw = join_df(df_raw, stores_t, ['store_nbr', 'date'])
print(len(df_raw))
df_raw.head()

127970257


,id,date,store_nbr,item_nbr,unit_sales,onpromotion,dcoilwtico,Holiday,family,class,perishable,city,state,StoreType,cluster,transactions
0,0,2013-01-01,25,103665,7.0,NaN,NaN,Holiday,BREAD/BAKERY,2712,1,Salinas,Santa Elena,D,1.0,770.0
1,1,2013-01-01,25,105574,1.0,NaN,NaN,Holiday,GROCERY I,1045,0,Salinas,Santa Elena,D,1.0,770.0
2,2,2013-01-01,25,105575,2.0,NaN,NaN,Holiday,GROCERY I,1045,0,Salinas,Santa Elena,D,1.0,770.0
3,3,2013-01-01,25,108079,1.0,NaN,NaN,Holiday,GROCERY I,1030,0,Salinas,Santa Elena,D,1.0,770.0
4,4,2013-01-01,25,108701,1.0,NaN,NaN,Holiday,DELI,2644,1,Salinas,Santa Elena,D,1.0,770.0


In [47]:
df_raw.dtypes

id                       int64
date            datetime64[ns]
store_nbr                 int8
item_nbr                 int32
unit_sales             float32
onpromotion             object
dcoilwtico             float16
Holiday                 object
family                  object
class                    int64
perishable               int64
city                    object
state                   object
StoreType               object
cluster                float64
transactions           float64
dtype: object

In [26]:
len(df_raw['class'].unique()), len(df_raw['family'].unique())

(334, 33)

In [27]:
df_raw['cluster'].max(), df_raw['transactions'].max(), df_raw['class'].max(), df_raw['perishable'].max(), df_raw['item_nbr'].max()

(17.0, 8359.0, 7780, 1, 2127114)

In [28]:
df_raw['class'].min(), df_raw['perishable'].min()

(1002, 0)

In [26]:
df_final.perishable.unique()

array([1, 0])

In [31]:
df_raw['cluster'].unique(), len(df_raw['cluster'].unique())

(array([ 1., 13.,  8.,  9.,  4.,  6., 15.,  7.,  3., 12., 16., 10.,  2.,  5., 11., 14., 17., nan]),
 18)

In [48]:
convert_dict = {'class': 'int16',
               'perishable': 'int8',
               'cluster': 'float16',
               'transactions': 'float16'}
df_raw = df_raw.astype(convert_dict)

In [35]:
df_raw.columns

Index(['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion',
       'dcoilwtico', 'Holiday', 'city', 'state', 'StoreType', 'cluster',
       'transactions', 'family', 'class', 'perishable'],
      dtype='object')

In [29]:
df_raw[df_raw['cluster'].isnull()]

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,dcoilwtico,Holiday,family,class,perishable,city,state,StoreType,cluster,transactions
7208276,7164318,2013-06-19,10,103501,1.386294,True,98.25000,NaN,CLEANING,3008,0,NaN,NaN,NaN,NaN,NaN
7208277,7164319,2013-06-19,10,105576,2.397895,True,98.25000,NaN,GROCERY I,1045,0,NaN,NaN,NaN,NaN,NaN
7208278,7164320,2013-06-19,10,106716,2.302585,True,98.25000,NaN,GROCERY I,1032,0,NaN,NaN,NaN,NaN,NaN
7208279,7164321,2013-06-19,10,108701,0.693147,True,98.25000,NaN,DELI,2644,1,NaN,NaN,NaN,NaN,NaN
7208280,7164322,2013-06-19,10,108786,3.367296,True,98.25000,NaN,CLEANING,3044,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94025110,92197366,2016-09-27,23,2027090,0.693147,False,44.65625,NaN,GROCERY I,1060,0,NaN,NaN,NaN,NaN,NaN
94025111,92197367,2016-09-27,23,2027252,0.693147,False,44.65625,NaN,GROCERY I,1034,0,NaN,NaN,NaN,NaN,NaN
94025112,92197368,2016-09-27,23,2029494,0.693147,False,44.65625,NaN,BEVERAGES,1122,0,NaN,NaN,NaN,NaN,NaN
94025113,92197369,2016-09-27,23,2031273,1.098612,False,44.65625,NaN,PRODUCE,2026,1,NaN,NaN,NaN,NaN,NaN


In [49]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127970257 entries, 0 to 127970256
Data columns (total 16 columns):
id              int64
date            datetime64[ns]
store_nbr       int8
item_nbr        int32
unit_sales      float32
onpromotion     object
dcoilwtico      float16
Holiday         object
family          object
class           int16
perishable      int8
city            object
state           object
StoreType       object
cluster         float16
transactions    float16
dtypes: datetime64[ns](1), float16(3), float32(1), int16(1), int32(1), int64(1), int8(2), object(6)
memory usage: 10.7+ GB


In [27]:
df_raw['date'].max(), df_raw['date'].min()

(Timestamp('2017-08-15 00:00:00'), Timestamp('2013-01-01 00:00:00'))

In [26]:
df_raw.loc[df_raw['date'] == '2016-08-15']

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,dcoilwtico,Holiday,family,class,perishable,city,state,StoreType,cluster,transactions
89869949,88042205,2016-08-15,1,103665,0.693147,False,45.71875,Holiday,BREAD/BAKERY,2712,1,Quito,Pichincha,D,13.0,1776.0
89869950,88042206,2016-08-15,1,105574,0.693147,False,45.71875,Holiday,GROCERY I,1045,0,Quito,Pichincha,D,13.0,1776.0
89869951,88042207,2016-08-15,1,105575,2.995732,False,45.71875,Holiday,GROCERY I,1045,0,Quito,Pichincha,D,13.0,1776.0
89869952,88042208,2016-08-15,1,105577,0.693147,False,45.71875,Holiday,GROCERY I,1045,0,Quito,Pichincha,D,13.0,1776.0
89869953,88042209,2016-08-15,1,105693,0.693147,False,45.71875,Holiday,GROCERY I,1034,0,Quito,Pichincha,D,13.0,1776.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89966398,88138654,2016-08-15,54,2026893,1.098612,False,45.71875,Holiday,GROCERY I,1034,0,El Carmen,Manabi,C,3.0,883.0
89966399,88138655,2016-08-15,54,2026945,0.693147,False,45.71875,Holiday,GROCERY I,1034,0,El Carmen,Manabi,C,3.0,883.0
89966400,88138656,2016-08-15,54,2026983,1.791759,False,45.71875,Holiday,GROCERY I,1034,0,El Carmen,Manabi,C,3.0,883.0
89966401,88138657,2016-08-15,54,2027090,1.098612,False,45.71875,Holiday,GROCERY I,1060,0,El Carmen,Manabi,C,3.0,883.0


In [27]:
len(df_raw)

127970257

In [50]:
df_final = df_raw[87000000:]

In [51]:
df_final.shape

(40970257, 16)

In [52]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40970257 entries, 87000000 to 127970256
Data columns (total 16 columns):
id              int64
date            datetime64[ns]
store_nbr       int8
item_nbr        int32
unit_sales      float32
onpromotion     object
dcoilwtico      float16
Holiday         object
family          object
class           int16
perishable      int8
city            object
state           object
StoreType       object
cluster         float16
transactions    float16
dtypes: datetime64[ns](1), float16(3), float32(1), int16(1), int32(1), int64(1), int8(2), object(6)
memory usage: 3.4+ GB


In [54]:
df_raw=None
df = None
for f in table_names: f=None
for g in table_names1: f=None
gc.collect()

161

In [55]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper gfunction that adds columns relevant to a date."
    fld = df[fldname] # contains the column
    if drop: df.drop(fldname, axis=1, inplace=True)
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64
    
    if not np.issubdtype(fld_dtype, np.datetime64): 
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    convert_dict = {'Year': 'int16',
               'Month': 'int8',
               'Week' : 'int8',
               'Day' : 'int8',
               'Dayofweek': 'int8',
               'Dayofyear' : 'int16',
               }
    df = df.astype(convert_dict)
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    return df

In [56]:
%%time 
df_final = add_datepart(df_final, 'date')

CPU times: user 43.4 s, sys: 4.9 s, total: 48.3 s
Wall time: 48.3 s


In [57]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40970257 entries, 87000000 to 127970256
Data columns (total 28 columns):
id                  int64
store_nbr           int8
item_nbr            int32
unit_sales          float32
onpromotion         object
dcoilwtico          float16
Holiday             object
family              object
class               int16
perishable          int8
city                object
state               object
StoreType           object
cluster             float16
transactions        float16
Year                int16
Month               int8
Week                int8
Day                 int8
Dayofweek           int8
Dayofyear           int16
Is_month_end        bool
Is_month_start      bool
Is_quarter_end      bool
Is_quarter_start    bool
Is_year_end         bool
Is_year_start       bool
Elapsed             int64
dtypes: bool(6), float16(3), float32(1), int16(3), int32(1), int64(2), int8(6), object(6)
memory usage: 4.0+ GB


In [75]:
%%time
df_final.to_csv(index=False)

MemoryError: 

In [37]:
df_final.head()

,id,store_nbr,item_nbr,unit_sales,onpromotion,dcoilwtico,Holiday,family,class,perishable,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
87000000,85272711,29,1489899,2.397895,False,NaN,NaN,PRODUCE,2028,1,...,17,6,199,False,False,False,False,False,False,1468713600
87000001,85272712,29,1490588,1.098612,False,NaN,NaN,BABY CARE,6022,0,...,17,6,199,False,False,False,False,False,False,1468713600
87000002,85272713,29,1501508,1.785741,False,NaN,NaN,PRODUCE,2012,1,...,17,6,199,False,False,False,False,False,False,1468713600
87000003,85272714,29,1501520,2.197225,False,NaN,NaN,PRODUCE,2002,1,...,17,6,199,False,False,False,False,False,False,1468713600
87000004,85272715,29,1501521,3.988984,False,NaN,NaN,PRODUCE,2004,1,...,17,6,199,False,False,False,False,False,False,1468713600


In [42]:
df_final['Elapsed'].max()

1502755200

In [48]:
len(df_final['city'].unique())

23

In [59]:
len(df_final['perishable'].unique())

2

In [66]:
df_final.columns, len(df_final.columns)

(Index(['id', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion',
        'dcoilwtico', 'Holiday', 'family', 'class', 'perishable', 'city',
        'state', 'StoreType', 'cluster', 'transactions', 'Year', 'Month',
        'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end',
        'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end',
        'Is_year_start', 'Elapsed'],
       dtype='object'), 28)

In [65]:
#len(df_final['famliy'].unique()), 
len(df_final['class'].unique()) 

334

In [69]:
procs = [FillMissing, Categorify, Normalize]
cat_names = ['store_nbr', 'onpromotion', 'Holiday', '' 'Year', 'city', 'state', 'StoreType', 'cluster', 'perishable', 'class',
       'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'family',
       'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end',
       'Is_year_start']
cont_names = ['id', 'item_nbr', 'transactions', 'dcoilwtico', 'Elapsed']
dep_var = 'unit_sales'

In [70]:
len(cat_names)+len(cont_names)+1

28

In [71]:
valid_idxs = range(int(len(df_final) *0.9), len(df_final))
valid_idxs

range(36873231, 40970257)

In [72]:
len(df_final)

40970257

In [ ]:
df_final =  pd.read_csv(f'{path}/data_final.csv', parse_dates=['date'],
                    dtype=types, infer_datetime_format=True)

In [73]:
data = TabularDataBunch.from_df(path, df_final, dep_var, cat_names=cat_names, cont_names=cont_names, procs=procs, valid_idx=valid_idxs)

In [74]:
data.show_batch(rows=10)

store_nbr,onpromotion,Holiday,Year,city,state,StoreType,cluster,perishable,class,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,family,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,transactions_na,dcoilwtico_na,id,item_nbr,transactions,dcoilwtico,Elapsed,target
6,False,#na#,2016,Quito,Pichincha,D,13.0,0,1387,8,31,4,3,217,False,"LIQUOR,WINE,BEER",False,False,False,False,False,False,False,-1.5664,0.4530,nan,nan,-1.5959,2.0
28,False,#na#,2016,Guayaquil,Guayas,E,10.0,0,1124,10,40,3,0,277,False,BEVERAGES,False,False,False,False,False,False,False,-1.0140,-0.1592,nan,nan,-1.0140,7.0
24,False,#na#,2017,Guayaquil,Guayas,D,1.0,0,1026,3,10,6,0,65,False,GROCERY I,False,False,False,False,False,False,False,0.4673,-1.1004,nan,nan,0.4794,19.0
51,False,Event,2017,Guayaquil,Guayas,A,17.0,1,2166,5,19,14,6,134,False,DAIRY,False,False,False,False,False,False,True,1.1638,0.5587,nan,nan,1.1486,14.0
40,False,#na#,2016,Machala,El Oro,C,3.0,0,4138,11,47,26,5,331,False,PERSONAL CARE,False,False,False,False,False,False,True,-0.5100,1.4003,nan,nan,-0.4904,1.0
24,False,#na#,2016,Guayaquil,Guayas,D,1.0,0,3020,12,51,20,1,355,False,CLEANING,False,False,False,False,False,False,False,-0.2758,-0.6524,nan,nan,-0.2576,6.0
50,False,#na#,2016,Ambato,Tungurahua,A,14.0,0,1030,12,52,27,1,362,False,GROCERY I,False,False,False,False,False,False,False,-0.2088,0.5580,nan,nan,-0.1897,9.0
15,False,#na#,2017,Ibarra,Imbabura,C,15.0,0,3038,1,2,15,6,15,False,CLEANING,False,False,False,False,False,False,True,-0.0335,1.3501,nan,nan,-0.0055,3.0
45,False,Bridge,2016,Quito,Pichincha,A,11.0,0,4141,11,44,4,4,309,False,PERSONAL CARE,False,False,False,False,False,False,False,-0.7149,0.1623,nan,nan,-0.7037,1.0
4,False,Holiday,2016,Quito,Pichincha,D,9.0,0,1032,7,29,23,5,205,False,GROCERY I,False,False,False,False,False,False,True,-1.6794,-1.4257,nan,nan,-1.7123,1.0


In [76]:
df_final =None
gc.collect()

446

In [77]:
data

TabularDataBunch;

Train: LabelList (36873231 items)
x: TabularList
store_nbr 29; onpromotion False; Holiday #na#; Year 2016; city Guayaquil; state Guayas; StoreType E; cluster 10.0; perishable 1; class 2028; Month 7; Week 28; Day 17; Dayofweek 6; Dayofyear 199; Is_month_end False; family PRODUCE; Is_month_start False; Is_quarter_end False; Is_quarter_start False; Is_year_end False; Is_year_start False; transactions_na False; dcoilwtico_na True; id -1.7295; item_nbr 0.6132; transactions nan; dcoilwtico nan; Elapsed -1.7705; ,store_nbr 29; onpromotion False; Holiday #na#; Year 2016; city Guayaquil; state Guayas; StoreType E; cluster 10.0; perishable 0; class 6022; Month 7; Week 28; Day 17; Dayofweek 6; Dayofyear 199; Is_month_end False; family BABY CARE; Is_month_start False; Is_quarter_end False; Is_quarter_start False; Is_year_end False; Is_year_start False; transactions_na False; dcoilwtico_na True; id -1.7295; item_nbr 0.6144; transactions nan; dcoilwtico nan; Elapsed -1.7705; ,stor

In [55]:
data.save('data_final.pkl')

MemoryError: 

In [ ]:
data = load_data(path, 'data_final.pkl')

In [80]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy).to_fp16()